# **(Resolvido) Exercícios: Não Comparecimento em Consultas**
**Autor:** [Anderson França](https://www.linkedin.com/in/anderson-m-franca/) | **Contato:** [github.com/andfranca](https://github.com/andfranca/estatistica-e-aprendizado-de-maquinas-ptbr)

<a href="https://creativecommons.org/licenses/by/4.0/deed.en"><img align="left" width="80" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc.png"/></a>

Imagine que você trabalha na equipe de gestão de um hospital e recebeu um desafio importante: **reduzir o número de pacientes que marcam consultas, mas não comparecem**.

As ausências impactam diretamente o funcionamento da unidade de saúde, gerando desperdício de recursos, aumento no tempo de espera para outros pacientes e dificuldades no planejamento da equipe médica.

Agora, surge a pergunta: **é possível prever quais pacientes têm maior risco de faltar à consulta?**

O conjunto de dados **naocomparecimento.csv**, contém 110.527 registros de consultas médicas, com 14 variáveis associadas a cada paciente e sua consulta. A principal variável a ser analisada indica se o paciente compareceu ou não à consulta.


##**Base de dados**

A variável alvo é `No-show`, que indica se o paciente faltou à consulta (`Yes`) ou compareceu (`No`).


Existem 14 variáveis, incluindo:
- Data da consulta
- Local do atendimento
- Características do paciente (idade, gênero, presença de doenças crônicas)
- Se recebeu lembrete via SMS
- Se estava em um programa de assistência social (Bolsa Família - Scholarship)
- Quantos dias de antecedência a consulta foi marcada

## **Tradução das Variáveis**

| Inglês      | Português      |
|-------------------|---------------------|
| PatientId        | ID do Paciente       |
| AppointmentID    | ID da Consulta       |
| Gender          | Gênero               |
| ScheduledDay    | Data do Agendamento  |
| AppointmentDay  | Data da Consulta     |
| Age            | Idade                 |
| Neighbourhood  | Bairro                |
| Scholarship    | Bolsa Família         |
| Hipertension   | Hipertensão           |
| Diabetes       | Diabetes              |
| Alcoholism     | Alcoolismo            |
| Handcap        | Deficiência           |
| SMS_received   | SMS Recebido          |
| No-show       | Não Comparecimento    |



## Contexto

Essa base permite analisar fatores que influenciam a ausência dos pacientes, ajudando unidades de saúde a reduzir faltas e melhorar o atendimento.

### Possíveis hipóteses para investigação:
- Pacientes mais jovens ou mais velhos faltam mais?
- O tempo entre o agendamento e a consulta influencia na presença?
- Receber um lembrete por SMS reduz faltas?
- Pacientes com certas condições médicas faltam mais?

## Roteiro de Análise

- Realize a análise exploratória univariada de todas as variáveis no conjunto de dados, interpretando suas distribuições.
- Faça a análise bidimensional entre cada variável explicativa e a variável resposta. Quais variáveis parecem ter maior influência sobre a variável de interesse?
- Para as variáveis categóricas, converta-as para o tipo category do pandas.
- Para este estudo, exclua as variáveis ID, Neighbourhood, e data da análise.
- Construa um modelo de árvore de decisão usando o scikit-learn para prever a variável resposta.
- Obtenha as classificação para cada observação no conjunto de dados.
- Gere a tabela de classificação para avaliar o desempenho do modelo e encontre o melhor ponto de corte para classificar as probabilidades. - Qual é o percentual de classificações corretas? E quais são a precision e recall do modelo?

In [ ]:
# Importar a biblioteca pandas para manipulação de dados
import pandas as pd

In [ ]:
# Carregar a base de dados a partir de um arquivo CSV online
db_nao_comparecimento = pd.read_csv('https://raw.githubusercontent.com/andfranca/proadi-sus-ciencia-de-dados-ia/refs/heads/main/bases/naocomparecimento.csv')

Vamos inspecionar a estrutura desse `dataFrame`

In [ ]:
#Visualizar estrutura dos dados
db_nao_comparecimento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


**Resumo**
- Número total de registros: 110.527
- Número de colunas: 14
- Valores nulos: Nenhuma coluna contém valores nulos.

**Tipos de dados:**
- **int64 (números inteiros):** 9 colunas
- **object (strings ou datas):** 5 colunas
- **float (decimais)**: 1 coluna


**Observações:**
- As colunas `ScheduledDay` e `AppointmentDay` estão como object, então precisam ser convertidas para datetime para análises temporais.
- A coluna `PatientId` pode ser irrelevante para o modelo, pois não contribui para a predição.
- A variável `No-show` já está completa e sem valores nulos, o que facilita a modelagem.

In [ ]:
db_nao_comparecimento.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


Vamos remover as colunas `PatientId` e `AppointmentID`, pois essas informações são apenas identificadores dos pacientes e das consultas, não agregando valor preditivo ao nosso modelo. Como queremos trabalhar apenas com variáveis que possam influenciar o não comparecimento, essa limpeza é necessária.


In [ ]:
# Remover colunas de identificação
db_nao_comparecimento.drop(['PatientId', 'AppointmentID'], axis=1, inplace=True)

Podemos ver que a base tem duas colunas de datas que podem ser bem úteis para a análise: `ScheduledDay` (quando a consulta foi marcada) e `AppointmentDay` (quando ela realmente acontece).

Para facilitar cálculos e manipulações, vamos converter essas colunas para o formato datetime, assim conseguimos trabalhar melhor com elas depois.

In [ ]:
#converter datas para formato datetime
db_nao_comparecimento['ScheduledDay'] = pd.to_datetime(db_nao_comparecimento['ScheduledDay'])
db_nao_comparecimento['AppointmentDay'] = pd.to_datetime(db_nao_comparecimento['AppointmentDay'])

Agora que as datas estão no formato certo, podemos calcular o tempo que passou entre o agendamento e o dia da consulta. Vamos criar a variável **DiasAgendamento**, que nos ajuda a entender se o tempo de espera influencia no não comparecimento. Isso pode ser um fator importante para prever quais pacientes têm maior chance de faltar.

In [ ]:
# Criar uma variável de dias entre o agendamento e a consulta
db_nao_comparecimento['DiasAgendamento'] = (db_nao_comparecimento['AppointmentDay'] - db_nao_comparecimento['ScheduledDay']).dt.days

Agora que já extraímos a informação que realmente importa (os dias entre o agendamento e a consulta), podemos remover as colunas `ScheduledDay` e `AppointmentDay`. Assim, deixamos a base mais enxuta e evitamos ter dados redundantes.

In [ ]:
# Remover as colunas de data após extrair a informação relevante
db_nao_comparecimento.drop(['ScheduledDay', 'AppointmentDay'], axis=1, inplace=True)

Existem algumas inconsistências na base, como consultas com **datas menores que a data de agendamento** e **idades negativas**. Como não conhecemos muito bem a origem desses dados e não conseguimos corrigir esses erros diretamente, vamos simplesmente filtrar essas linhas para evitar problemas na análise.

In [ ]:
# Remover observações com valores inconsistentes
db_nao_comparecimento = db_nao_comparecimento[db_nao_comparecimento['DiasAgendamento'] >= 0]
db_nao_comparecimento = db_nao_comparecimento[db_nao_comparecimento['Age'] >= 0]

# Análise Exploratória de Dados

Fizemos uma análise exploratória simples para entender melhor os dados e garantir que não há problemas óbvios, como valores inconsistentes ou colunas desnecessárias. No entanto, não vamos nos aprofundar muito nessa etapa, pois nosso foco aqui é a construção do modelo para fins didáticos.

Vale destacar que, em um cenário real, uma análise exploratória mais detalhada é essencial para identificar padrões, verificar distribuições e tratar possíveis vieses nos dados. Quanto melhor entendermos os dados, melhores serão as decisões na modelagem e os resultados do modelo.

# Pré-processamento de dados

Depois de realizar a análise exploratória, entender melhor a base de dados e criar novas variáveis, estamos prontos para seguir para a etapa de modelagem. Mas antes disso, precisamos converter algumas variáveis categóricas em valores numéricos usando Label Encoding, garantindo que o modelo consiga interpretá-las corretamente.

In [ ]:
# Carregar o label encoder
from sklearn.preprocessing import LabelEncoder

Antes de partir para a modelagem, precisamos transformar algumas variáveis categóricas em valores numéricos.

- Usamos Label Encoding para converter `Gender` e `Neighbourhood`, já que modelos de machine learning não conseguem lidar diretamente com texto.
- Também transformamos a variável `No-show`, onde "Yes" (não compareceu) vira `1` e "No" (compareceu) vira `0`.

In [ ]:
# Aplicar Label Encoding para converter variáveis categóricas em numéricas
db_nao_comparecimento['Gender'] = LabelEncoder().fit_transform(db_nao_comparecimento['Gender'])
db_nao_comparecimento['Neighbourhood'] = LabelEncoder().fit_transform(db_nao_comparecimento['Neighbourhood'])

# Converter a variável alvo para valores numéricos (1 = Não compareceu, 0 = Compareceu)
db_nao_comparecimento['No-show'] = db_nao_comparecimento['No-show'].map({'Yes': 1, 'No': 0})

In [ ]:
#Visualizar as colunas do banco de dados
db_nao_comparecimento.columns

Index(['Gender', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'No-show',
       'DiasAgendamento'],
      dtype='object')

Agora vamos separar nossa base em variáveis preditoras (X) e variável alvo (y).

- X contém as características dos pacientes e das consultas, que serão usadas para tentar prever o não comparecimento.
- y é a variável que queremos prever, ou seja, se o paciente faltou (`1`) ou compareceu (`0`).

In [ ]:
# Definir as variáveis preditoras (X) e a variável alvo (y)
X = db_nao_comparecimento[[
                          'Gender',
                          'Age',
                          'Neighbourhood',
                          'Scholarship',
                          'Hipertension',
                          'Diabetes',
                          'Alcoholism',
                          'Handcap',
                          'SMS_received',
                          'DiasAgendamento'
                          ]]
y = db_nao_comparecimento['No-show']

Agora vamos dividir os dados em treino e teste.

- **70% dos dados serão usados para treinar** o modelo e encontrar padrões.
- **30% serão usados para testar o modelo**, avaliando sua capacidade de prever corretamente os não comparecimentos.
- Definimos um `random_state=42` para garantir que a divisão seja sempre a mesma, permitindo a replicação dos resultados.

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=42)

# Modelagem: Regressão Logística

Sabemos que a nossa variável alvo é binária (paciente falta = `1` ou comparece = `0`), então vamos começar testando a **Regressão Logística**, pois é um modelo simples, eficiente e interpretável para esse tipo de problema. Além disso, ela nos ajuda a entender a influência de cada variável na previsão, treina rapidamente e serve como um bom *benchmark* inicial antes de irmos pra modelos mais complexos.

Se o desempenho não for tão bom, podemos explorar outras abordagens, como árvores de decisão ou outros modelos.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

Vamos começar com um modelo simples: Regressão Logística.

- Definimos um `random_state=42` para garantir replicação.
- Ajustamos `max_iter=1000` para garantir que o modelo tenha tempo suficiente para convergir.
- Usamos `class_weight='balanced'`, pois a base pode ter desbalanceamento entre os pacientes que comparecem e os que faltam, ajudando a evitar viés do modelo.

In [ ]:
# definir os parâmetros do modelo de Regressão Logística
reg_log = LogisticRegression(random_state=42,
                             max_iter=1000,
                             class_weight='balanced')

# Treinar o modelo
reg_log.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

Agora que treinamos o modelo, vamos usá-lo para fazer previsões no conjunto de teste (`X_test`). Isso nos permitirá comparar os valores previstos pelo modelo com os valores reais (`y_test`) e avaliar seu desempenho. É como se estivéssemos testando o modelo no mundo real, mas com a vantagem de já sabermos as respostas!

In [ ]:
# Gerar a previsão do modelo em uma base nova (X_test)
y_pred_log = reg_log.predict(X_test)

Vamos dar uma olhada na distribuição da variável alvo na base de treino antes de avaliarmos o modelo. Como utilizamos `class_weight='balanced'`, é importante entender se os dados estão desbalanceados. Caso haja uma grande diferença entre as classes (por exemplo, muito mais pacientes que comparecem do que os que faltam), isso pode impactar a performance do modelo. Isso nos ajudará a interpretar melhor os resultados.

In [ ]:
# Verificar a distribuição da variável alvo no conjunto de treino
y_train.value_counts(normalize=True)

,proportion
No-show,
0,0.714955
1,0.285045


- 71,5% dos pacientes compareceram à consulta (0).
- 28,5% dos pacientes faltaram (1).

Isso mostra que temos um certo desbalanceamento na base, já que a maioria dos pacientes comparece. Esse desbalanceamento pode impactar a performance do modelo, fazendo com que ele tenha uma tendência maior a prever que o paciente comparecerá.

Como já utilizamos `class_weight='balanced'` na regressão logística, o modelo tentará compensar esse desbalanceamento.

Agora vamos avaliar as métricas do modelo

In [ ]:
print(metrics.classification_report(y_test, y_pred_log))

              precision    recall  f1-score   support

           0       0.76      0.55      0.64     15424
           1       0.34      0.57      0.42      6164

    accuracy                           0.55     21588
   macro avg       0.55      0.56      0.53     21588
weighted avg       0.64      0.55      0.58     21588



O modelo não está indo tão bem, principalmente para prever os pacientes que faltam. A acurácia geral é de 55%, mas quando olhamos os detalhes:

- Ele acerta 76% das previsões de quem comparece, mas só 34% de quem falta.
- O `recall` para faltas está um pouco melhor (57%), mas ainda longe do ideal.
- O `F1-score` para faltas ficou em 42%, ou seja, não está identificando bem esses casos.

Provavelmente isso acontece porque a base está desbalanceada (muito mais gente comparece do que falta) e a regressão logística é um modelo simples. Para tentar melhorar, poderíamos testar algumas abordagens, como:

- Fazer uma seleção de variáveis usando o **p-valor** para manter apenas as mais relevantes.
- **Analisar os coeficientes** do modelo para entender quais variáveis estão realmente impactando a previsão.
- **Alterar a base de dados**, talvez gerando novas variáveis ou tentando balancear melhor as classes.


Mas, antes de ajustar esses detalhes, vamos testar um modelo mais robusto, como uma **Árvore de Decisão**, para ver se conseguimos melhorar o desempenho!

# Árvore de Decisão
Agora vamos testar uma Árvore de Decisão, que é um modelo mais flexível e pode capturar melhor padrões não lineares nos dados.

- Vamos começar com `class_weight='balanced'` para lidar com o desbalanceamento da base.
- Escolhemos `entropy` como critério para a divisão dos nós, priorizando informações mais puras.
- Vamos limitar a profundidade da árvore a 5 para evitar que o modelo fique muito complexo e dê *overfitting*.
- Definimos `min_samples_leaf=1` e `min_samples_split=2`, permitindo que a árvore cresça com poucos dados por nó.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Criar e treinar um modelo de Árvore de Decisão
ar = DecisionTreeClassifier(class_weight='balanced',
                            criterion= 'entropy',
                            max_depth = 5,
                            min_samples_leaf = 1,
                            min_samples_split = 2,
                            random_state=42)
ar.fit(X_train, y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=5, random_state=42)

Agora que treinamos a Árvore de Decisão, vamos usá-la para fazer previsões nos dados de teste (X_test). Assim, poderemos comparar os resultados com o modelo anterior e ver se conseguimos melhorar a performance na identificação dos pacientes que faltam às consultas.

In [ ]:
# Gerar previsões com o modelo de Árvore de Decisão
y_pred_ar = ar.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, y_pred_ar))

              precision    recall  f1-score   support

           0       0.77      0.56      0.65     15424
           1       0.34      0.58      0.43      6164

    accuracy                           0.57     21588
   macro avg       0.56      0.57      0.54     21588
weighted avg       0.65      0.57      0.59     21588



#### **Análise do resultado da Árvore**

O modelo de Árvore de Decisão apresentou um leve aumento na acurácia para 57% (antes era 55% com a Regressão Logística), mas vamos olhar os detalhes:

**- Para quem compareceu (`0`):**

  - **Precisão:** 77% (quando o modelo prevê que alguém compareceu, ele acerta 77% das vezes).
  - **Recall:** 56% (identificou corretamente 56% dos que realmente compareceram).

- **Para quem faltou (`1`):**

  - **Precisão:** 34% (quando o modelo prevê uma falta, só 34% das previsões estavam certas).
  - **Recall:** 58% (melhor do que antes, agora acerta mais casos de quem faltou).


- **F1-score:**

  - Para faltas (`1`): 43% (um pequeno ganho em relação ao modelo anterior).


### Conclusão
A Árvore de Decisão teve um desempenho um pouco melhor, principalmente no recall para quem faltou, ou seja, está conseguindo detectar mais casos de ausência. Porém, a precisão para prever faltas ainda é baixa.

## **Sugestões de Melhorias para os próximos passos**

### **Ajustar os hiperparâmetros**
Podemos testar diferentes valores para os parâmetros da árvore e ver se encontramos uma configuração melhor:

- **max_depth:** Aprofundar ou reduzir a árvore pode ajudar a equilibrar viés e variância.
- **min_samples_split** e **min_samples_leaf:** Ajustar esses valores pode evitar que a árvore fique muito complexa ou restritiva.
- **criterion:** Testar "gini" em vez de "entropy" pode afetar a forma como a árvore faz as divisões.

### **Seleção de variáveis**

Nem todas as variáveis podem estar ajudando o modelo. Podemos:

- Analisar a importância das variáveis na árvore e remover as menos relevantes.
- Criar novas variáveis a partir das já existentes para capturar melhor os padrões.

### **Balanceamento da base**
Mesmo usando class_weight='balanced', podemos testar técnicas mais avançadas para lidar com o desbalanceamento:

- **Oversampling (SMOTE):** Aumentar o número de casos de pacientes que faltaram.
- **Undersampling:** Reduzir o número de casos de quem compareceu para equilibrar as classes.

### **Feature Engineering**
Podemos criar novas variáveis para tentar capturar melhor os padrões de não comparecimento, como:

- Dia da semana da consulta (pacientes podem faltar mais em certos dias).
- Tempo entre o agendamento e a consulta (faixas) para identificar períodos mais críticos.
- Interação entre variáveis, como condições médicas e idade.

### **Ajuste do ponto de corte**
Atualmente, o modelo classifica como 1 (faltou) quando a probabilidade é maior que 50%, mas podemos testar outros `limiares de decisão` para ver se conseguimos aumentar o recall sem perder muita precisão.

## Extras
`Os temas a seguir não foram abordados durante as aulas aindas.`

In [ ]:
from sklearn.model_selection import GridSearchCV

Agora vou testar diferentes configurações para a Árvore de Decisão e ver se consego melhorar o desempenho do modelo.

- max_depth: Testamos árvores mais rasas (5) e mais profundas (10 e 15).
- min_samples_split: Ajustamos a quantidade mínima de amostras para um nó ser dividido, o que pode impactar o aprendizado.
- min_samples_leaf: Definimos limites para o tamanho mínimo de cada folha da árvore, evitando divisões muito pequenas.
- criterion: Vamos comparar Gini e Entropia para ver qual critério gera melhores divisões nos dados.

In [ ]:
# Definir os hiperparâmetros para ajuste da Árvore de Decisão
param_grid = {
    'max_depth': [5, 10, 15],  # Testar diferentes profundidades da árvore
    'min_samples_split': [2, 5, 10], # Número mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4], # Número mínimo de amostras em uma folha
    'criterion': ['gini', 'entropy'] # Testar os dois critérios de divisão
}

Aqui vou usar o `GridSearchCV` para encontrar a melhor combinação de hiperparâmetros para a Árvore de Decisão para não ter que testar todas as combinações manualmente.

- Criei um modelo base ar e deixei os hiperparâmetros em aberto para serem ajustados.
- O GridSearchCV testa todas as combinações definidas no param_grid, aplicando validação cruzada com 5 divisões (cv=5).
- Usei `scoring='f1'` para otimizar o modelo baseado na métrica F1-score, que equilibra precisão e recall.
- `O n_jobs=1` define que o processo será executado em um único núcleo, mas daria para aumentar para acelerar o treinamento em máquinas mais potentes.

In [ ]:
# Criar o modelo base de Árvore de Decisão
ar = DecisionTreeClassifier(class_weight='balanced',
                            random_state=42)

# Aplicar GridSearchCV para encontrar os melhores hiperparâmetros
ar_gs = GridSearchCV(estimator=ar, param_grid=param_grid, cv=5, scoring='f1', n_jobs=1)
ar_gs.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=42),
             n_jobs=1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 10, 15],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='f1')

In [ ]:
ar_gs.best_params_

{'criterion': 'entropy',
 'max_depth': 5,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

O `GridSearchCV` testou várias combinações de hiperparâmetros e encontrou os melhores valores para a Árvore de Decisão:

- **criterion = 'entropy':** O modelo teve um desempenho melhor usando a entropia para definir as divisões nos nós.
- **max_depth = 5:** Uma árvore com profundidade 5 foi suficiente para capturar padrões sem overfitting.
- **min_samples_leaf = 1:** Nós folha podem ter apenas 1 amostra, permitindo maior granularidade.
- **min_samples_split = 2:** Um nó é dividido sempre que há pelo menos 2 amostras, o que mantém flexibilidade no crescimento da árvore.

In [ ]:
# Gerar previsões com o modelo de Árvore de Decisão
y_pred_ar_gs = ar_gs.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, y_pred_ar_gs))

              precision    recall  f1-score   support

           0       0.77      0.56      0.65     15424
           1       0.34      0.58      0.43      6164

    accuracy                           0.57     21588
   macro avg       0.56      0.57      0.54     21588
weighted avg       0.65      0.57      0.59     21588



## Análise do resultado final da Árvore de Decisão otimizada:

Mesmo após ajustar os hiperparâmetros com `GridSearchCV`, os resultados da árvore continuam muito próximos da versão anterior:

- Acurácia geral: 57% (mesmo valor da árvore sem otimização).
- Para quem compareceu (0):
  - Precisão: 77% (quando prevê que alguém compareceu, acerta 77% das vezes).
  - Recall: 56% (identificou corretamente 56% dos que realmente compareceram).
- Para quem faltou (1):
  - Precisão: 34% (quando prevê uma falta, acerta só 34% das vezes).
  - Recall: 58% (melhor na detecção de quem faltou).

- F1-score:
  - 43% para quem faltou, um resultado semelhante ao modelo sem ajuste.

# Conclusão:
Mesmo ajustando os hiperparâmetros, não conseguimos um grande ganho de desempenho. Isso indica que o modelo pode estar limitado pelos dados disponíveis, ou seja, pode ser que faltem variáveis mais relevantes para capturar padrões de não comparecimento.

Se quisermos melhorar ainda mais, poderíamos testar:
- **Criar novas variáveis** (como dia da semana, horário da consulta, sazonalidade, etc.).
- **Técnicas de balanceamento de classes** (como SMOTE para aumentar os exemplos de faltas).
- **Testar um modelo mais complexo**, como Random Forest ou XGBoost, para capturar relações mais profundas nos dados.